The purpose of this notebook is to manipulate the dataframe I created in "NFL_ImportandClean_Fanduel. <br><br>
My main goal is to add the player position to make filtering easier and also possibly see if the correlation in difference between player value and price the next week.

In [1]:
#pulling in all the necessary python libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from tabulate import tabulate
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler

In [2]:
#create frame from saved csv
nfl_frame = pd.read_csv('nfl_weekly_fanduel_scores.csv')
#rename columns
nfl_frame.columns = ['Row','Week','Name','Team','Opponent','Fanduel_Points','Fanduel_Price','Game Location']

In [3]:
#removing extra index column
del nfl_frame['Row']

My first goal is to add player position to the table. Looking at the table, I know that it is sorted in ascending order by Week (1->16) and then in descending order by Fanduel_Points. The positions are arrayed as QB->RB->WR->TE->K->Defense using this ordering, resetting each time the week changes.

I'm going to try to write a comprehension that starts with Quarterback, and then continues checking to see if Fanduel_Points is <= the previous row. If yes, It will remain quarterback. If no, then it will check for Week number. If week number is the same, then it will move to the next choice in the list (RB) and continue going through the comprehension until Fanduel_Points is less than the previous row. It will then run the same check for receiver, TE,K,Def until that week number changes, at which point it will begin again at QB if I do it right, this will let me add the posiitons every week without any manual work beyond running the script.

In [5]:
nfl_frame

,Week,Name,Team,Opponent,Fanduel_Points,Fanduel_Price,Game Location,Position
0,1,"Wilson, Russell",sea,atl,31.78,8400.0,Away,QB
1,1,"Rodgers, Aaron",gnb,min,30.76,7600.0,Away,QB
2,1,"Allen, Josh",buf,nyj,28.18,7900.0,Home,QB
3,1,"Jackson, Lamar",bal,cle,27.50,9300.0,Home,QB
4,1,"Murray, Kyler",ari,sfo,27.30,7700.0,Away,QB
...,...,...,...,...,...,...,...,...
1297,3,Washington,was,cle,1.00,3900.0,Away,QB
1298,3,Jacksonville,jac,mia,0.00,3800.0,Home,QB
1299,3,New York J,nyj,ind,0.00,3800.0,Away,QB
1300,3,New York G,nyg,sfo,-2.00,3600.0,Home,QB


In [4]:
#creating position column I will add my comprehension to, with a default of QB
nfl_frame['Position'] = 'QB'

*This is your attempt at writing a column with if then statements to directly add a position column. You're not having a ton of success and are looking for help from Stack Overflow*

In [6]:
#writing comprehension to perform task listed above
def add_position(df):
    #creating list of my positions
    Positions = list(['QB','RB','WR','TE','K','Def'])
    #setting index as 0 to start with QB
    Role = 0
    #loop checking for week number and points scored
    for i in range(1,len(df)):
        #if week for new row matches week for last row, and points for new row <= last row, then same position as last row
        if df.Week[i]==df.Week[i-1] and df.Fanduel_Points[i]<= df.Fanduel_Points[i-1]:
            return Positions[Role]
        #if week for new row matches week for last row, and points for new row > last row, then next position in list
        elif df.Week[i] == df.Week[i-1] and df.Fanduel_Points[i] > df.Fanduel_Points[i-1]:
            Role +=1
            return Positions[Role]
        #if new week, reset and begin from Positions[0] to begin labeling as QB again
        elif df.Week[i] > df.Week[i-1] :
            Role = 0
            return Positions[Role]


In [7]:
nfl_frame['Position'] = nfl_frame.apply(add_position(nfl_frame))

AttributeError: 'DataFrame' object has no attribute 'QB'